Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/tutorials/img-classification-part1-training.png)

In [8]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

import azureml.core
from azureml.core import Workspace

# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.0.57


# Create the AMLS workspace...

## You will be asked to log into Azure and be given a code in the output message area to enter.

## Workspace Properties from Above:

Name | Description
---- | -----------
name            | A name you chose to call the workspace.  We'll use the value aready in the code.
subscription_id | The id of the subscription the workspace will be assigned to.  You can get this from the Azure portal.
resource_group  | A name you want all the Azure resourced creates for the workspace to be associated with.  Makes rsource management easier.
location | Azure data center location closest to you that support creation of AMLS workspaces.  

## To make reconnecting to this workspace in future notebooks easier, save the configuration setting using the code in the cell below.

### Connect to workspace in future work...

In the future, we can use the code below to connect back to this workspace. 
The code creates a workspace object from the existing workspace. `Workspace.from_config()` reads the file **config.json** and loads the details into an object named `ws`.  You don't need to use this now since we are still connected from when we created the workspace but this will come in handy later.

In [90]:
# load workspace configuration from the config.json file in the current folder.
ws = Workspace.from_config()
print(ws.name, ws.location, ws.resource_group, ws.location, sep='\t')

bpcdp100ws	eastus2	rg_bpcdp100ws	eastus2


### Create experiment

Create an experiment to track the runs in your workspace. A workspace can have muliple experiments. 

In [10]:
experiment_name = 'dp100class'

from azureml.core import Experiment

exp = Experiment(workspace=ws, name=experiment_name)
print('Experiment created or referenced if already there.')

Experiment created or referenced if already there.


### Create or Attach existing compute resource
By using Azure Machine Learning Compute, a managed service, data scientists can train machine learning models on clusters of Azure virtual machines. Examples include VMs with GPU support. In this tutorial, you create Azure Machine Learning Compute as your training environment. The code below creates the compute clusters for you if they don't already exist in your workspace.

**Creation of compute takes approximately 5 minutes.** If the AmlCompute with that name is already in your workspace the code will skip the creation process.

### Display a few rows of data to make sure the load worked.

_______________________

# Lab 2: Part B Deploy the model

Now, you're ready to deploy the model as a web service in Azure Container Instances (ACI). A web service is an image, in this case a Docker image, that encapsulates the scoring logic and the model itself.

In this part of the lab, you use Azure Machine Learning service (Preview) to:

Set up your testing environment
Retrieve the model from your workspace
Test the model locally
Deploy the model to ACI
Test the deployed model

ACI is not ideal for production deployments, but it is great for testing and understanding the workflow. For scalable production deployments, consider using AKS.

### Retrieve the model

You registered a model in your workspace previously. Assuming you needed to come back later to deploy the model, load this workspace and download the model to your local directory.

In [11]:
#  It remembers the pickle file name and downloads the model to that name.

from azureml.core import Workspace
from azureml.core.model import Model

ws = Workspace.from_config()

model=Model(ws, 'bwmodel')
model.download(target_dir='.', exist_ok=True)

import os 
# verify the downloaded model file
os.listdir()

# print('Complete')

['notebook_train_a_model_with_AMLS-demo.ipynb',
 'score.py',
 '.azureml',
 'AWBikeSales.csv',
 'dp100',
 'set_subscriptionid.ipynb',
 'Starting_Lab2.ipynb',
 'data',
 'birthweightemodel.pkl',
 'Starting_Lab1.ipynb',
 'birthwt.csv',
 '.ipynb_checkpoints',
 'notebook_deploy_and_use_a_model_demo.ipynb',
 'bwmodel.yml',
 'README.md',
 'notebook_train_a_model_with_AMLS.ipynb']

## Deploy as web service

Once you've tested the model and are satisfied with the results, deploy the model as a web service hosted in ACI. 

To build the correct environment for ACI, provide the following:
* A scoring script to show how to use the model
* An environment file to show what packages need to be installed
* A configuration file to build the ACI
* The model you trained before

### Create scoring script

Create the scoring script, called score.py, used by the web service call to show how to use the model.

You must include two required functions into the scoring script:
* The `init()` function, which typically loads the model into a global object. This function is run only once when the Docker container is started. 

* The `run(input_data)` function uses the model to predict a value based on the input data. Inputs and outputs to the run typically use JSON for serialization and de-serialization, but other formats are supported.

In [12]:
%%writefile score.py
import json
import numpy as np
import os
import pickle
from sklearn.externals import joblib
from sklearn.linear_model import LogisticRegression

from azureml.core.model import Model

def init():
    global model
    # retreive the path to the model file using the model name
    model_path = Model.get_model_path('bwmodel')
    model = joblib.load(model_path)

def run(raw_data):
    data = np.array(json.loads(raw_data)['data'])
    # make prediction
    y_hat = model.predict(data)
    # you can return any data type as long as it is JSON-serializable
    return y_hat.tolist()

Overwriting score.py


### Create environment file

Next, create an environment file, called myenv.yml, that specifies all of the script's package dependencies. This file is used to ensure that all of those dependencies are installed in the Docker image. This model needs `scikit-learn` and `azureml-sdk`.

In [36]:
## Using add_conda_package, you can only add one package at a time.

In [13]:
from azureml.core.conda_dependencies import CondaDependencies 

bwmodelenv = CondaDependencies()
bwmodelenv.add_conda_package('scikit-learn')
bwmodelenv.add_conda_package('pandas')
# bwmodelenv.add_conda_package(['scikit-learn', 'pandas', 'numpy'])

with open("bwmodel.yml","w") as f:
    f.write(bwmodelenv.serialize_to_string())
    
    
print('Complete')    

Complete


Review the content of the `myenv.yml` file.

In [14]:
with open("bwmodel.yml","r") as f:
    print(f.read())
    
print('Complete')    

# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2

- pip:
    # Required packages for AzureML execution, history, and data preparation.
  - azureml-defaults

- scikit-learn
- pandas
channels:
- conda-forge

Complete


### Create configuration file

Create a deployment configuration file and specify the number of CPUs and gigabyte of RAM needed for your ACI container. While it depends on your model, the default of 1 core and 1 gigabyte of RAM is usually sufficient for many models. If you feel you need more later, you would have to recreate the image and redeploy the service.

In [15]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               tags={"data": "lowbirthwt",  "method" : "sklearn"}, 
                                               description='Predict low birth weight with sklearn')

print('Complete')

Complete


### Deploy in ACI
Estimated time to complete: **about 7-8 minutes**

Configure the image and deploy. The following code goes through these steps:

1. Build an image using:
   * The scoring file (`score.py`)
   * The environment file (`myenv.yml`)
   * The model file
1. Register that image under the workspace. 
1. Send the image to the ACI container.
1. Start up a container in ACI using the image.
1. Get the web service HTTP endpoint.

In [16]:
model

Model(workspace=Workspace.create(name='bpcdp100ws', subscription_id='050aedf0-1ce2-495f-a726-be8756f5ab18', resource_group='rg_bpcdp100ws'), name=bwmodel, id=bwmodel:7, version=7, tags={}, properties={})

In [17]:
%%time
from azureml.core.webservice import Webservice
from azureml.core.image import ContainerImage

# configure the image
image_config = ContainerImage.image_configuration(execution_script="score.py", 
                                                  runtime="python", 
                                                  conda_file="bwmodel.yml")

CPU times: user 1.08 ms, sys: 249 µs, total: 1.33 ms
Wall time: 54.1 ms


In [18]:
image_config

In [19]:
%%time

from azureml.core.webservice import Webservice
from azureml.core.image import ContainerImage

service = Webservice.deploy_from_model(workspace=ws,
                                       name='sklearn-birthwt',
                                       deployment_config=aciconfig,
                                       models=[model],
                                       image_config=image_config)

service.wait_for_deployment(show_output=True)

print('Complete')

Creating image
Running.................................
Succeeded
Image creation operation finished for image sklearn-birthwt:5, operation "Succeeded"
Creating service
Running...............
SucceededACI service creation operation finished, operation "Succeeded"
Complete
CPU times: user 1.03 s, sys: 163 ms, total: 1.19 s
Wall time: 4min 22s


In [20]:
print('Done')

Done


Get the scoring web service's HTTP endpoint, which accepts REST client calls. This endpoint can be shared with anyone who wants to test the web service or integrate it into an application.

# Ways to get help with image creation issues...

In [39]:
# if you already have the image object handy
# print(service.image_build_log_uri)

# if you only know the name of the image (note there might be multiple images with the same name but different version number)
print(ws.images['sklearn-birthwt'].image_build_log_uri)

# list logs for all images in the workspace
#for name, img in ws.images.items():
#    print(img.name, img.version, img.image_build_log_uri)

None


In [40]:
print(service.scoring_uri)

http://d5790dea-bd70-4d58-ad66-416086e7702f.eastus2.azurecontainer.io/score


## Test deployed service

Earlier you scored all the test data with the local version of the model. Now, you can test the deployed model by just passing the test data we used earlier, X_test.  We're doing this in the interest of time as it will serve us well just to test the deployed service.  In a real use case, you might want to specially prepare some input data.  

The following code goes through these steps:
1. Send the data as a JSON array to the web service hosted in ACI. 

1. Use the SDK's `run` API to invoke the service. You can also make raw calls using any HTTP tool such as curl.

1. Print the returned predictions and plot them along with the input images. Red font and inverse image (white on black) is used to highlight the misclassified samples. 


In [21]:
import pandas as pd

df_features = pd.read_csv(r'./data/birthwt.csv')

In [80]:
from sklearn.model_selection import train_test_split
from sklearn import tree
import numpy as np
import pandas as pd

df_y = df_features['low']
df_lbw_model_test = df_features.drop(['low','bwt'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(df_lbw_model_test, df_y, test_size=0.2)

X_score = X_test.values
Y_score = y_test.values

X_score.shape

(38, 9)

In [4]:
print('done')

done


### Display the first few values to see that the call worked.

In [82]:
import json

test_samples = json.dumps({"data": X_score.tolist()})
test_samples = bytes(test_samples, encoding='utf8')

# predict using the deployed model
result = service.run(input_data=test_samples)

print('Complete')

Complete


In [83]:
result[0:5]

[0, 1, 0, 1, 0]

In [89]:
import requests
import json

# send a random row from the test set to score
random_index = np.random.randint(0, len(X_score)-1)
input_data = "{\"data\": [" + str(list(X_score[random_index])) + "]}"

headers = {'Content-Type':'application/json'}

# for AKS deployment you'd need to the service key in the header as well
# api_key = service.get_key()
# headers = {'Content-Type':'application/json',  'Authorization':('Bearer '+ api_key)} 

resp = requests.post(service.scoring_uri, input_data, headers=headers)

print("POST to url", service.scoring_uri)

print("input data:", input_data)
print("label:", Y_score[random_index])
print("prediction:", resp.text)

print('Complete')

POST to url http://62bedad3-bd0f-41b0-ae28-69c73811bcae.eastus2.azurecontainer.io/score
input data: {"data": [[181, 19, 105, 3, 0, 0, 0, 0, 0]]}
label: 0
prediction: [0]
Complete


You can also send raw HTTP request to test the web service.  Again, we are using the X_test data for convenience.

## Clean up resources

To keep the resource group and workspace for other tutorials and exploration, you can delete only the ACI deployment using this API call:

In [ ]:
# Uncomment the line below and run the code to clean up the resources.

# service.delete()